In [198]:
import keras
import numpy as np
from numpy import array
from numpy import hstack

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers.core import Masking

In [199]:
rawdata = pd.read_csv('Rnndata3.csv')

In [200]:
rawdata.head(5)

,time,Month,Day,Mon,Tue,Wed,Thu,Fri,Sat,Sun,...,ID15,ID16,ID17,ID18,ID19,ID20,ID21,ID22,ID23,ID24
0,0,12,1,0,0,0,0,1,0,0,...,9237,7323,4038,7457,7528,7797,10251,14608,9685,5399
1,1,12,2,0,0,0,0,0,1,0,...,1217,792,457,912,854,1099,1333,1917,1465,802
2,2,12,3,0,0,0,0,0,0,1,...,346,233,134,302,244,270,480,723,546,234
3,3,12,4,1,0,0,0,0,0,0,...,14363,11470,6148,10930,10442,12212,14989,21759,14615,7956
4,4,12,5,0,1,0,0,0,0,0,...,10356,8327,4763,8327,7159,8786,11431,16993,10956,5994


In [201]:
rawdata.shape

(365, 43)

In [202]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, 18:]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [203]:
seq_raw = rawdata.values

In [204]:
x, y = split_sequences(seq_raw, 4)

In [205]:
y

array([[6743, 8690, 9985, ..., 16993, 10956, 5994],
       [6134, 8152, 9815, ..., 15728, 10637, 5490],
       [6188, 7848, 9928, ..., 15774, 10112, 5491],
       ...,
       [6704, 8255, 11864, ..., 17916, 12382, 6362],
       [6224, 7844, 10976, ..., 17817, 12301, 6403],
       [5806, 7549, 10480, ..., 16106, 11191, 5884]], dtype=object)

In [206]:
x.shape

(361, 4, 43)

In [207]:
y.shape

(361, 25)

In [208]:
class LossHistory(keras.callbacks.Callback):
    def init(self):
        self.losses=[]
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [209]:
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(4, 43)))
model.add(LSTM(150, activation='relu', return_sequences=True, init = 'glorot_normal', input_shape=(4, 43)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(25))
model.compile(optimizer='adam', loss='mse')

C:\Users\Chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(150, activation="relu", return_sequences=True, input_shape=(4, 43), kernel_initializer="glorot_normal")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [210]:
x_train = x[:-31]
y_train = y[:-31]

In [211]:
x_train.shape

(330, 4, 43)

In [212]:
y_train.shape

(330, 25)

In [213]:
x_test = x[:31]
y_test = y[:31]

In [214]:
x_test.shape

(31, 4, 43)

In [215]:
y_test.shape

(31, 25)

In [219]:
model.fit(x_train, y_train, epochs=2**13, batch_size = 31)

Epoch 1/8192
330/330 [==============================] - 0s 1ms/step - loss: 1797080.6176
Epoch 2/8192
330/330 [==============================] - 0s 732us/step - loss: 1828045.7657
Epoch 3/8192
330/330 [==============================] - 0s 697us/step - loss: 1845244.1631
Epoch 4/8192
330/330 [==============================] - 0s 726us/step - loss: 1834218.0508
Epoch 5/8192
330/330 [==============================] - 0s 1ms/step - loss: 1925929.1803
Epoch 6/8192
330/330 [==============================] - 0s 1ms/step - loss: 1769194.4629
Epoch 7/8192
330/330 [==============================] - 0s 809us/step - loss: 1724708.8042
Epoch 8/8192
330/330 [==============================] - 0s 736us/step - loss: 1719860.1403
Epoch 9/8192
330/330 [==============================] - 0s 729us/step - loss: 1649076.0589
Epoch 10/8192
330/330 [==============================] - 0s 733us/step - loss: 1648566.2277
Epoch 11/8192
330/330 [==============================] - 0s 765us/step - loss: 1765725.1602
Epo

In [220]:
scores= model.evaluate(x_test, y_test)
print(scores)

31/31 [==============================] - 0s 352us/step
93000.640625


In [221]:
a = model.predict(x_train)